**I. Descripción de Necesidades de Limpieza**


- Para iniciar se necesita cambiar la codificación de los datos ya que la Ñ aparece con un simbolo extraño y las tildes igual remplazandolas por n y su letra sin tilde respectivamente

- Luego se necesitan llenar los valores nulos, si son de texto quedarian "Desconocidos" y si son número quedan en 0 ya que no hay información de cuantos hubo en su momento o eran lineas que no habian sido creadas en su momento

- mapeamos una columna nueva llamada "CLIMA" donde pondremos los meses secos y lluviosos de la ciudad de Medellín repartidos de la siguiente manera enero, febrero, diciembre, junio, julio y agosto (secos) y marzo, abril, mayo septiembre, octubre y noviembre (lluviosos)

- mapeamos los eventos mas importantes de medellín, siendo enero - año nuevo, julio - feria de flores, agosto - feria de flores, septiembre - feria de la antioqueñidad, noviembre - alumbrados navideños y diciembre - navidad y alumbrados y a los meses no mencionados se llenaran sus nulos con "Ningún evento importante" y las ñ serán reemplazadas por n para evitar cualquier error

- finalizamos cambiando el nombre del dataset por Pasajeros_movilizados(new) para saber que es el dataset nuevo con sus modificaciones

**II. Limpieza y Transformación de Datos**

Importación de librerias necesarias

In [47]:
!pip install chardet ftfy #Necesario para reemplazar los datos mal codificados en la base de datos

Lectura de la base de datos

In [54]:
import pandas as pd
df = pd.read_csv('Pasajeros_movilizados.csv')

Organización de datos mal escritos en la descarga

In [55]:
import chardet
from ftfy import fix_text

# 1. Detectar codificación real del archivo
with open('Pasajeros_movilizados.csv', 'rb') as f:
    rawdata = f.read()
    encoding = chardet.detect(rawdata)['encoding']
    print(f"Codificación detectada: {encoding}")

# 2. Leer el archivo con la codificación detectada
try:
    df = pd.read_csv('Pasajeros_movilizados.csv', encoding=encoding)
except UnicodeDecodeError:
    # Si falla, probar con otras codificaciones comunes
    for enc in ['latin1', 'cp1252', 'utf-8', 'utf-16']:
        try:
            df = pd.read_csv('Pasajeros_movilizados.csv', encoding=enc)
            print(f"Éxito con codificación: {enc}")
            break
        except:
            continue

# 3. Corregir nombres de columnas (manualmente si es necesario)
nuevos_nombres = {
    'A�O': 'ANO',
    'SEM_A�O': 'SEM_ANO',
    'L�NEA_O': 'LINEA_O',
    'A�O-MES': 'ANO_MES'
}
df = df.rename(columns=nuevos_nombres)

# 4. Corregir texto en todas las celdas (opcional pero recomendado)
df = df.applymap(lambda x: fix_text(x) if isinstance(x, str) else x)

print("¡Archivo corregido exitosamente!")
print("Nombres de columnas actuales:", df.columns.tolist())

Codificación detectada: utf-8
¡Archivo corregido exitosamente!
Nombres de columnas actuales: ['ANO', 'SEMESTRE', 'NUM_MES', 'MES', 'SEM_ANO', 'L-A_PAX_MOV', 'L-B_PAX_MOV', 'L-K_PAX_MOV', 'L-J_PAX_MOV', 'L-H_PAX_MOV', 'L-M_PAX_MOV', 'L-L_PAX_MOV', 'L-TA_PAX_MOV', 'L-1_PAX_MOV', 'L-2_PAX_MOV', 'C-3_PAX_MOV', 'C6_PAX_MOV', 'LINEA_O', 'ANO_MES', 'ORDEN']


<ipython-input-55-c0275f83a338>:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: fix_text(x) if isinstance(x, str) else x)


Limpieza de datos

In [56]:
#Mes en N/A por desconocido
df['NUM_MES'] = df['NUM_MES'].fillna('Desconocido')
df['MES'] = df['MES'].fillna('Desconocido')


#Cambiar N/A por 0 en las lineas de metro
# Identificar las columnas de pasajeros (todas las que terminan con _PAX_MOV)
pax_columns = [col for col in df.columns if col.endswith('_PAX_MOV')]

# Reemplazar valores nulos con 0 en estas columnas
df[pax_columns] = df[pax_columns].fillna(0)
df['LINEA_O'] = df['LINEA_O'].fillna('0')

# Verificar que ya no hay nulos en estas columnas
print(df[pax_columns].isnull().sum())

L-A_PAX_MOV     0
L-B_PAX_MOV     0
L-K_PAX_MOV     0
L-J_PAX_MOV     0
L-H_PAX_MOV     0
L-M_PAX_MOV     0
L-L_PAX_MOV     0
L-TA_PAX_MOV    0
L-1_PAX_MOV     0
L-2_PAX_MOV     0
C-3_PAX_MOV     0
C6_PAX_MOV      0
dtype: int64


Agregar nuevas filas
Clima


In [57]:
# Diccionario para mapear meses a clima
condiciones_clima = {
    'enero': 'seco',
    'febrero': 'seco',
    'diciembre': 'seco',
    'junio': 'seco',
    'julio': 'seco',
    'agosto': 'seco',
    'marzo': 'lluvia',
    'abril': 'lluvia',
    'mayo': 'lluvia',
    'septiembre': 'lluvia',
    'octubre': 'lluvia',
    'noviembre': 'lluvia'
}

# Crear la nueva columna 'Clima' basada en el diccionario
df['CLIMA'] = df['MES'].map(condiciones_clima).fillna('Desconocido')

df['CLIMA'] = df['MES'].map(condiciones_clima)

# Verificar el resultado
print(df[['MES', 'CLIMA']].head(10))

           MES   CLIMA
0  Desconocido     NaN
1  Desconocido     NaN
2  Desconocido     NaN
3  Desconocido     NaN
4        enero    seco
5      febrero    seco
6        marzo  lluvia
7        abril  lluvia
8         mayo  lluvia
9        junio    seco


Agregar nuevas filas evento

In [58]:
# Diccionario de eventos por mes (ejemplos basados en Medellín/Colombia)
eventos_por_mes = {
    'enero': 'ano nuevo',
    'julio': 'feria de las flores',
    'agosto': 'feria de las flores',
    'septiembre': 'feria de la antioquenidad',
    'noviembre': 'alumbrados navidenos',
    'diciembre': 'navidad y alumbrados'
}

# Crear la columna 'Evento' basada en el diccionario
df['EVENTO'] = df['MES'].map(eventos_por_mes)

# Opcional: Rellenar meses sin evento con "Ninguno" o otro valor
df['EVENTO'] = df['EVENTO'].fillna('ninguno importante')

# Verificar el resultado
print(df[['MES', 'EVENTO']].head(12))

            MES               EVENTO
0   Desconocido   ninguno importante
1   Desconocido   ninguno importante
2   Desconocido   ninguno importante
3   Desconocido   ninguno importante
4         enero            ano nuevo
5       febrero   ninguno importante
6         marzo   ninguno importante
7         abril   ninguno importante
8          mayo   ninguno importante
9         junio   ninguno importante
10        julio  feria de las flores
11       agosto  feria de las flores


Nuevo dataset

In [59]:
df.to_csv('Pasajeros_movilizados(new).csv', index=False)

**III. Validación:**

Los cambios de este dataset han sido basicos, reemplazo de nulos y la creación de nuevas columnas, que nos ayudaran para las respuestas, de las preguntas en la actividad 1, logrando que se mantenga la misma granularidad de los datos y sus datos anteriores completamente limpios y sin errores *texto en cursiva*